In [2]:
from manim import *
import math

## Background Color


In [8]:
%%manim -v WARNING -qm BgColor

class BgColor(Scene):
    def construct(self):
        self.camera.background_color = BLUE
        self.add(Circle().set_fill(RED, opacity=1))
        self.wait()

## Moving Camera Scene (2D)

The 2D camera can be moved in the XY plane relative to the scene, and can be saved/restored.

In [12]:
%%manim -v WARNING -qm MovingCameraScene

class MovingCameraScene(MovingCameraScene):
    def construct(self):
        self.add(Circle().set_fill(RED, opacity=1))
        self.wait()
        
        self.play(self.camera.frame.animate.move_to(LEFT*3)) # scene goes RIGHT
        
        self.camera.frame.save_state()
        self.play(self.camera.frame.animate.move_to(RIGHT*3))
        
        self.play(Restore(self.camera.frame))
        
        self.play(self.camera.frame.animate.rotate(PI))

## 3D Camera

`move_camera()` (the animated version) and `set_camera_orientation()` (the static version) have a lot of params for setting the following:
  * polar coordinates of the camera
  * cartesian coordiantes of center of the scene
  * zoom level of the scene

In [18]:
%%manim -v WARNING -qm ThreeDCamera

class ThreeDCamera(ThreeDScene):
    def construct(self):
        self.add(Circle().set_fill(RED, opacity=1))
        self.wait()
        
        self.move_camera(phi=PI) # animated already (can take other animations as array)
        self.wait()
        
        self.add_fixed_in_frame_mobjects(Circle().move_to(UP))  # something that doesn't move with the camera
        self.move_camera(phi=0) # the non-animated version is set_camera_orientation()
        

## 3D Camera Orientation

These params apply to `set_camera_orientation()` and `move_camera()` both.

Note that __ommited params are left alone__ so that you don't have to keep respecifying them.  You can tell that will happen by the `None` defaults in the docs.

Summary

  * the angles maintain the origin as the center of the frame
  * the angles can be applied together or separate for the same result
  * the direction and initial values of the angles aren't always what you'd expect (see examples)
  * zooming is along the axis to the origin after applying the angles
    * focal distance, on the other hand, just adds shape distortions (if not the default of 20)
  * the frame center is treated as the origin for angles and zooming
    * the angles are always applied after re-centering
      * that means that if you do nothing but re-center, you will get some rotation as well
  * camera orientaiton is applied in such a way that you can set the fields 1 by 1 or altogether in any order and get the same end result
  * unrecognized arguments pass through as keyword args even if they do nothing
    * this is important because in the 3blue1brown video, he uses args that used to be in the equivalent method
    * if you directly copy them, they do nothing, so you need to translate them to do something

In [51]:
%%manim -v WARNING -qm ThreeDCamera2

class ThreeDCamera2(ThreeDScene):
    def construct(self):
        # Plotting some axes that look different from each other
        axes = ThreeDAxes(
            x_range=(-10, 10),
            y_range=(-2, 2, 1),
            z_range=(-0, 100, 10),
        )
        self.add(axes)
        
        # Put a cube in the middle
        self.add(Cube())
        
        # Accessing current values
        camera = self.renderer.camera
        print('phi', 'theta', 'gamma', 'zoom', 'frame_center', 'focal_distance')
        print(camera.phi, math.degrees(camera.theta), camera.gamma, camera.zoom, 
              camera.frame_center, camera.focal_distance)
        
        # Phi (angle with z-axis)
        self.move_camera(phi=math.radians(90), run_time=5) # 90 degrees from start
        self.set_camera_orientation(phi=math.radians(0)) # initially 0
        self.wait()
        
        # Theta (spinning around z-axis)
        self.move_camera(theta=math.radians(0), run_time=5) # 90 degrees from start
        self.set_camera_orientation(theta=math.radians(-90)) # initially -90 degrees
        self.wait()
        
        # Gamma (spinning camera around axis to origin)
        self.move_camera(gamma=math.radians(90), run_time=5) # 90 degrees from start
        self.set_camera_orientation(gamma=0)  # initially 0
        self.wait()
        
        # All 3 angles together (equivalent to the above)
        # 30 degrees tilted up (and origin re-centered)
        # 45 degrees around the scene (scene goes clockwise because camera goes counter)
        # 20 degrees tilted to the side (scene goes counter because camera goes clockwise)
        # NOTE: still looking at origin after all this no matter what angles you do
        self.move_camera(phi=math.radians(30), theta=math.radians(-45), gamma=math.radians(20), run_time=5)
        self.set_camera_orientation(phi=0, theta=math.radians(-90), gamma=0)
        self.move_camera(phi=math.radians(30), run_time=5)
        self.move_camera(theta=math.radians(-45), run_time=5)
        self.move_camera(gamma=math.radians(20), run_time=5)
        self.set_camera_orientation(phi=0, theta=math.radians(-90), gamma=0)
        self.wait()
        
        # Zoom (factor, along axis to origin)
        self.set_camera_orientation(phi=math.radians(30), theta=math.radians(-45), gamma=math.radians(20), run_time=5)
        self.move_camera(zoom=0.5, run_time=3)
        self.move_camera(zoom=2, run_time=3)
        self.move_camera(zoom=1, run_time=3)
        self.set_camera_orientation(phi=0, theta=math.radians(-90), gamma=0)
        self.wait()
        
        # Frame Center (effective origin)
        self.move_camera(frame_center=3*(IN+LEFT), run_time=3)
        self.move_camera(gamma=math.radians(90))
        self.move_camera(frame_center=ORIGIN, gamma=0, run_time=3)
        self.move_camera(gamma=math.radians(90), phi=math.radians(30), frame_center=3*(IN+LEFT), run_time=3)
        self.move_camera(frame_center=ORIGIN, run_time=3)
        self.set_camera_orientation(gamma=0, phi=0, frame_center=ORIGIN)
        self.wait()
        
        # Focal Distance
        self.move_camera(focal_distance=5, run_time=3) # wide angle
        self.move_camera(focal_distance=40, run_time=3) # telephoto
        self.move_camera(focal_distance=1, run_time=3) # super wide angle
        self.move_camera(focal_distance=20, run_time=3) # the default
        
        # Arguments that do nothing
        self.move_camera(gamma=0.1, center=3*(IN+LEFT), run_time=3) # center does not set frame_center
        print(camera.frame_center)
        self.move_camera(gamma=0.2, width=100, run_time=3)
        self.wait()

phi theta gamma zoom frame_center focal_distance
0 -90.0 0 1 [0. 0. 0.] 20.0


Animation 23: _MethodAnimation(ValueTracker):   0%|     | 0/1 [00:00<?, ?it/s]/Users/davidpetrofsky/miniforge3/envs/ai/lib/python3.10/site-packages/manim/camera/three_d_camera.py:321: RuntimeWarning: divide by zero encountered in divide
  factor = focal_distance / (focal_distance - zs)
/Users/davidpetrofsky/miniforge3/envs/ai/lib/python3.10/site-packages/manim/mobject/types/vectorized_mobject.py:1002: RuntimeWarning: invalid value encountered in double_scalars
  if abs(p0[0] - p1[0]) > atol + rtol * abs(p1[0]):
/Users/davidpetrofsky/miniforge3/envs/ai/lib/python3.10/site-packages/manim/mobject/types/vectorized_mobject.py:1004: RuntimeWarning: invalid value encountered in double_scalars
  if abs(p0[1] - p1[1]) > atol + rtol * abs(p1[1]):


[0. 0. 0.]


## Animating Camera Movements

In [66]:
%%manim -v WARNING -qm ThreeDCamera3

class ThreeDCamera3(ThreeDScene):
    def construct(self):
        # Plotting some axes that look different from each other
        axes = ThreeDAxes(
            x_range=(-10, 10),
            y_range=(-2, 2, 1),
            z_range=(-0, 100, 10),
        )
        self.add(axes)
        
        # Put some shapes in the middle
        cube = Cube()
        self.add(cube)
        self.add(Dot3D(radius=1, color=RED).move_to(2*LEFT))
        self.add(Dot3D(radius=1, color=GREEN).move_to(2*RIGHT))
        
        # Static
        self.set_camera_orientation(phi=math.radians(30)) # applies right on 1st frame (whenver something animates)
        
        # Animated
        self.move_camera(theta=0, run_time=5, rate_func=linear)
        
        # With Other Animations
        self.move_camera(theta=math.radians(90), run_time=5, rate_func=linear, 
                         added_anims=[cube.animate.scale(0.5)])
        
        # Ambient
        self.begin_ambient_camera_rotation(rate=math.radians(90)/5, about='theta')
        self.play(cube.animate.scale(2), run_time=5, rate_func=linear)
        self.stop_ambient_camera_rotation(about='theta')
        
        self.wait()

## Camera Overlays

The `add_fixed_in_frame_mobjects()` method does the following:

  1. Add the objects to the scene if it's not already in there.
  1. Fix the object at its literal coordinates during camera rotations.
  
     * If the camera is already rotated, then the object will not be relative to the current rotation.
     * Basically just treat it as X-Y plane of the screen itself, ignoring the camera rotation.
  1. If removed from the list of fixed objects, it will jump to the camera-rotated position starting on the next frame and keep rotating.
  
  NOTE: if you want to __write-on__ overlay text, you can add it as an overlay first and then do the write-on, as shown below.  As long as no animations happen in between, you won't see it in the initial frame.

In [3]:
%%manim -v WARNING -qm ThreeDCamera4

class ThreeDCamera4(ThreeDScene):
    def construct(self):
        # Plotting some axes that look different from each other
        axes = ThreeDAxes(
            x_range=(-10, 10),
            y_range=(-2, 2, 1),
            z_range=(-0, 100, 10),
        )
        self.add(axes)
        
        # Put some shapes in the middle
        cube = Cube()
        self.add(cube)
        red = Dot3D(radius=1, color=RED).move_to(2*LEFT)
        self.add(red)
        green = Dot3D(radius=1, color=GREEN).move_to(2*RIGHT)
        self.add(green)
        
        # Start from an angle
        self.set_camera_orientation(phi=math.radians(45), theta=math.radians(45))
        
        # Fix the existing green sphere
        self.add_fixed_in_frame_mobjects(green)
        
        # Rotate
        self.move_camera(theta=math.radians(-45), rate_func=linear, run_time=5)
        
        # Add another fixed objects on the fly
        self.add_fixed_in_frame_mobjects(Dot3D(radius=1, color=PURPLE).move_to(UP*2))
        
        # Rotate again
        self.move_camera(theta=math.radians(-135), rate_func=linear, run_time=5)
        
        # Allow the green sphere to rotate again
        self.remove_fixed_in_frame_mobjects(green)
        
        # Rotate again
        self.move_camera(theta=math.radians(-225), rate_func=linear, run_time=5)
        
        # Add some overlay text
        text = Tex('Hello, world!')
        self.add_fixed_in_frame_mobjects(text)
        
        # Rotate again
        self.move_camera(theta=math.radians(-315), rate_func=linear, run_time=5)
        
        # Re-write the text during rotation
        self.move_camera(theta=math.radians(-405), rate_func=linear, run_time=5,
                         added_anims=[Write(text)])
        
        self.wait()

## Light Source

A `ThreeDScene` has a member called `self.camera.light_source` which is a point that can be moved around to create lighting effects.  It is not the primary light but just for shading effects.

In this example, the animation doesn't work too well - you see a dark spot (the light source) move smoothly, but the lighting changes very suddenly at the end of the movement.

It works much better when you use __OpenGL__.

In [15]:
%%manim -v WARNING -qm LightingScene

class LightingScene(ThreeDScene):
    def construct(self):
        cube = Cube(fill_opacity=1.0)
        sphere = Sphere().move_to(RIGHT*2)
        self.add(cube)
        self.add(sphere)
        
        self.set_camera_orientation(phi=math.radians(45), theta=math.radians(30))
        
        light = self.camera.light_source
        self.play(light.animate.move_to(LEFT * 3), run_time=3)
        self.play(light.animate.move_to(RIGHT * 10), run_time=3)

Manim Community v0.18.1

## Degrees Constant

You can multiply by this instead of using `math.radians()` when a function takes radians.

In [3]:
print(DEGREES)

0.017453292519943295
